<a href="https://colab.research.google.com/github/wxmiked/cannabis-deforestation/blob/feature-add-download-naip/notebooks/custom-raster-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cannabis Parcel Images for LabelMe

Mike Dvorak [mike@weathertactics.io](mailto:mike@weathertactics.io)

## Setup

First, we install TorchGeo and a couple of other dependencies for downloading data from Microsoft's Planetary Computer.

In [1]:
%pip install torchgeo planetary_computer pystac geopandas rasterio matplotlib

Note: you may need to restart the kernel to use updated packages.


## Imports

Next, we import TorchGeo and any other libraries we need.

In [2]:
import os
import tempfile
from urllib.parse import urlparse

import matplotlib.pyplot as plt
import planetary_computer
import pystac
import torch
from torch.utils.data import DataLoader

from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler, Units
from torchgeo.transforms import AppendNDBI, AppendNDVI, AppendNDWI

%matplotlib inline
plt.rcParams['figure.figsize'] = (4, 4)

## Downloading

Let's download some data to play around with. In this example, we'll create a dataset for loading Sentinel-2 images. Yes, TorchGeo already has a built-in class for this, but we'll use it as an example of the steps you would need to take to add a dataset that isn't yet available in TorchGeo. We'll show how to download a few bands of Sentinel-2 imagery from the Planetary Computer. This may take a few minutes.

In [3]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [4]:
time_range = "2016-01-01/2016-12-31"
bbox = [-120.56802294590112,38.307368125981206,-120.47440785318257,38.36284893862011]


search = catalog.search(collections=["naip"], bbox=bbox, datetime=time_range)
items = search.item_collection()
items

In [5]:
import os
from urllib.parse import urlparse
import tempfile

root = os.path.join(tempfile.gettempdir(), 'naip')
os.makedirs(root, exist_ok=True)  # Ensure the directory exists

for item in items:
    signed_item = planetary_computer.sign(item)
    url = signed_item.assets["image"].href
    filename = urlparse(url).path.split('/')[-1]
    
    # Check if file already exists
    filepath = os.path.join(root, filename)
    if not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        download_url(url, root, filename)
    else:
        print(f"Skipping {filename} - already exists")

Skipping m_3812045_sw_10_h_20160620.tif - already exists
Skipping m_3812045_nw_10_h_20160620.tif - already exists
Skipping m_3812044_sw_10_h_20160620.tif - already exists
Skipping m_3812044_se_10_h_20160620.tif - already exists
Skipping m_3812044_nw_10_h_20160620.tif - already exists
Skipping m_3812044_ne_10_h_20160620.tif - already exists


This downloads the following files:

In [6]:
print(root)
sorted(os.listdir(root))

/var/folders/56/gwh1m3s973l4jc97_wdr2qqw0000gn/T/naip


['m_3812044_ne_10_h_20160620.tif',
 'm_3812044_nw_10_h_20160620.json',
 'm_3812044_nw_10_h_20160620.tif',
 'm_3812044_se_10_h_20160620.tif',
 'm_3812044_sw_10_h_20160620.tif',
 'm_3812045_nw_10_h_20160620.tif',
 'm_3812045_sw_10_h_20160620.tif']

## Crop the images to cannabis parcels

We want to create a training images of cannabis growing operations (hoop houses and rows of plants) for an instance segmentation model that will classify new NAIP images.

In [16]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import os
from pathlib import Path
import re

# only pick up the NAIP files whose names contain a date
filename_glob  = 'm_*.tif'
filename_regex = r'^m_[0-9]{7}_[nsew]{2}_10_h_(?P<date>\d{8})'
date_pattern   = re.compile(filename_regex)

# Define paths
parcels_path = "../cannabis-parcels/cannabis-registry-2018-commercial-permits-parcels.gpkg"
output_dir = "../cannabis-parcels/cannabis-parcels-masked/"
naip_dir = "../data-to-import/microsoft/naip/calaveras-county-west-point-training-set"  # Assuming this is where your NAIP images are stored

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get NAIP CRS from first image
naip_crs = None
for naip_file in Path(naip_dir).glob('*.tif'):
    with rasterio.open(naip_file) as src:
        naip_crs = src.crs
        print(f"NAIP CRS: {naip_crs}")
        break

if naip_crs is None:
    raise ValueError("No NAIP images found to determine CRS")

# Read the parcels GeoPackage and reproject to NAIP CRS
parcels = gpd.read_file(parcels_path)
print(f"Original parcels CRS: {parcels.crs}")
parcels = parcels.to_crs(naip_crs)
print(f"Reprojected parcels to: {parcels.crs}")

# Filter for cannabis parcels
cannabis_parcels = parcels[parcels['cannabis_permit'] == 1]

# Function to process each parcel
def process_parcel(row):
    apn = row['APN']
    geometry = row['geometry']
    bounds = geometry.bounds
    minx, miny, maxx, maxy = bounds
    polygon_bounds = box(minx, miny, maxx, maxy)

    # Find matching NAIP image
    for naip_file in Path(naip_dir).glob(filename_glob):
        file_name = naip_file.name
        m = date_pattern.match(file_name)
        if not m:
            # skip any file that doesn't match your naming scheme
            continue

        date = m.group('date')   # e.g. '20160721'
        with rasterio.open(naip_file) as src:
            try:
                out_image, out_transform = mask(src, [polygon_bounds], crop=True, filled=True)
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width":  out_image.shape[2],
                    "transform": out_transform
                })

                # include both APN and date in the filename
                output_path = os.path.join(output_dir, f"{apn}_{date}.tif")
                with rasterio.open(output_path, "w", **out_meta) as dest:
                    dest.write(out_image)

                print(f"Processed parcel {apn} ({date})")
                break
            except ValueError:
                continue

# Process each cannabis parcel
for _, row in cannabis_parcels.iterrows():
    process_parcel(row)

print("Processing complete!")

NAIP CRS: EPSG:26910
Original parcels CRS: EPSG:4326
Reprojected parcels to: PROJCS["NAD83 / UTM zone 10N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26910"]]
Processed parcel 21022005 (20160620)
Processed parcel 21019030 (20160620)
Processed parcel 21010030 (20160620)
Processed parcel 16025150 (20160620)
Processed parcel 21010009 (20160620)
Processed parcel 21010046 (20160620)
Processed parcel 21001049 (20160620)
Processed parcel 21013029 (20160620)
Processed 